In [85]:
#数独アプリ
from tkinter import *
import numpy as np

#フィールドの大きさ設定(倍率)
size=2

#マスのオブジェクト
class Square:
    def __init__(self,row,col,size):
        self.number=0#マスに入れる数字
        self.e=Entry(width=2,font=('Verdana',10*size))#マスの設定
        self.e.place(x=(row+1)*30*size,y=(col+1)*30*size)#マスの位置
        self.candidate=[1,2,3,4,5,6,7,8,9]#マスに入る数字の候補
        self.anticand=[1,2,3,4,5,6,7,8,9]#マスの対立候補(マスに入らない数字の候補)

    #マスに入力した数字を取得
    def GetNum(self):
        if self.e.get()!='':
            self.number=int(self.e.get())
        else:
            self.number=0
            
    #候補の削除(①)
    def DeleteCandidate(self,other):
        if other.number!=0 and other.number in self.candidate:
            self.candidate.remove(other.number)
            
    #候補の削除(②～④)
    def DeleteCandidateB(self,listA):
        if self.number!=0 and self.number in listA:
            listA.remove(self.number)
            
    #対立候補の抽出
    def MakeAnticand(self,listA):
        #対立候補を出す
        self.anticand=list(set([1,2,3,4,5,6,7,8,9])-set(self.candidate))
        #ダブる部分のみ抽出
        return list(set(listA)&set(self.anticand))
        
        

#数独を解くアルゴリズム
def solveSudoku(self):
    global masu
    #ループ回数
    count=0
    #マスの数字取得
    for i in range(0,9):
        for j in range(0,9):
            masu[i][j].GetNum()
    #
    while(count<10):
        for i in range(0,9):
            for j in range(0,9):
                #値が入っていれば次のループへ
                if masu[i][j].number!=0:
                    continue
                    
                ###①候補を消す
                
                for k in range(0,9):
                    #縦の列を見て候補を消す
                    masu[i][j].DeleteCandidate(masu[k][j])
                        
                    #横の列を見て候補を消す
                    masu[i][j].DeleteCandidate(masu[i][k])
                
                #エリアを見て候補を消す
                for k in range(int(i/3)*3,int(i/3)*3+3):
                    for l in range(int(j/3)*3,int(j/3)*3+3):
                        masu[i][j].DeleteCandidate(masu[k][l])
                          
                            
                ###②縦の対立候補を見て数を確定させる
                #候補作成
                anticand_row=[1,2,3,4,5,6,7,8,9]
                
                #既に確定している数字は候補から除く
                for k in range(0,9):
                    masu[i][k].DeleteCandidateB(anticand_row)
                    
                for k in range(0,9):
                    #自分のマスと確定したマスは考えない
                    if k==j or masu[i][k].number!=0:
                        continue
                    anticand_row=masu[i][k].MakeAnticand(anticand_row)

                    
                ###③横の対立候補を見て数を確定させる
                anticand_col=[1,2,3,4,5,6,7,8,9]
                
                for k in range(0,9):
                    masu[k][j].DeleteCandidateB(anticand_col)

                for k in range(0,9):
                    if k==i or masu[k][j].number!=0:
                        continue
                    anticand_col=masu[k][j].MakeAnticand(anticand_col)

                    
                ###④エリアの対立候補を見て数を確定させる
                anticand_area=[1,2,3,4,5,6,7,8,9]

                for k in range(int(i/3)*3,int(i/3)*3+3):
                    for l in range(int(j/3)*3,int(j/3)*3+3):
                        masu[l][k].DeleteCandidateB(anticand_area)

                for k in range(int(i/3)*3,int(i/3)*3+3):
                    for l in range(int(j/3)*3,int(j/3)*3+3):
                        if (k==j and l==i) or masu[l][k].number!=0:
                            continue
                        anticand_area=masu[l][k].MakeAnticand(anticand_area)

                ###候補が1つになったら値を確定する
                if len(masu[i][j].candidate)==1:#①で確定した場合
                    masu[i][j].number=masu[i][j].candidate[0]
                elif len(anticand_row)==1:#②で確定した場合
                    masu[i][j].number=anticand_row[0]
                elif len(anticand_col)==1:#③で確定した場合
                    masu[i][j].number=anticand_col[0]
                elif len(anticand_area)==1:#④で確定した場合
                    masu[i][j].number=anticand_area[0]
        count+=1
        
        ###マスに取得した数字を出力
        for i in range(0,9):
            for j in range(0,9):
                if masu[i][j].e.get()=='':
                    if masu[i][j].number!=0:
                        masu[i][j].e.insert(END,masu[i][j].number)
    print('終了')

#フィールド作成

geotext=str(320*size)+'x'+str(350*size)
win=Tk()
win.title('suudoku')
win.geometry(geotext)

#エリアの描線
canvas=Canvas(win,width=320*size,height=350*size)
canvas.create_line(115*size,20*size,115*size,300*size,width=size)
canvas.create_line(205*size,20*size,205*size,300*size,width=size)
canvas.create_line(20*size,115*size,300*size,115*size,width=size)
canvas.create_line(20*size,205*size,300*size,205*size,width=size)
canvas.pack()

#ボタンの作成
Button=Button(text='数独を解く')
Button.bind("<Button-1>",solveSudoku)
Button.place(x=160*size-30,y=300*size+10)

#マスの生成
masu=np.empty((9,9),dtype=Square)
for i in range(0,9):
    for j in range(0,9):
        masu[i][j]=Square(i,j,size)

win.mainloop()
